### Treinando um Modelo para Responder Perguntas com o Dataset Amazon Titles

O dataset The Amazon Titles é uma coleção que contém consultas textuais reais de usuários, juntamente com os títulos e descrições de produtos relevantes da Amazon associados a essas buscas.

Para a realização deste trabalho, vamos utilizar o arquivo trn.json, focando especificamente nas colunas ***title***, que corresponde ao título do produto, e ***content***, que contém a sua descrição.

Link de acesso ao dataset: https://drive.google.com/file/d/12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK/view

Equipe:
- Iêsa Lobato
- Rilson Soares
- Ismael Costa
- Felipe Vieira


### Ambiente de execução

GPU A100: potência e a memória necessárias para realizar o fine-tuning do nosso modelo de forma eficiente

### Baixando bibliotecas

- transformers: biblioteca da Hugging Face que dá acesso a modelos pré-treinados

- datasets: usada para carregar e manipular datasets

- torch (PyTorch): principal biblioteca de machine learning usada neste trabalho

- accelerate: biblioteca da Hugging Face que otimiza o código PyTorch para rodar em diferentes hardwares como GPUs

- bitsandbytes: necessária para a técnica de quantização QLoRA, pois permite carregar modelos grandes em GPUs com menos memória

- trl (Transformer Reinforcement Learning): auxilia o processo de fine-tuning

- peft (Parameter-Efficient Fine-Tuning): auxilia as implementações da técnica QLoRA.

- gdown: ferramenta para baixar arquivos do Google Drive

In [ ]:
!pip install transformers datasets torch accelerate bitsandbytes trl peft gdown -q

### Baixando o Dataset

In [ ]:
import gdown
import pandas as pd
from datasets import load_dataset
from datasets import Dataset

#Baixando dataset
file_id = '12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK'
output_path = 'Amazon-Titles-1.3MM.zip'
gdown.download(id=file_id, output=output_path, quiet=False)

#Descompactando
!unzip -o Amazon-Titles-1.3MM.zip

Downloading...
From (original): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK
From (redirected): https://drive.google.com/uc?id=12zH4mL2RX8iSvH0VCNnd3QxO4DzuHWnK&confirm=t&uuid=c0de5249-254f-4718-9fe5-d7814b4c3c5d
To: /content/Amazon-Titles-1.3MM.zip
100%|██████████| 890M/890M [00:03<00:00, 245MB/s]


Archive:  Amazon-Titles-1.3MM.zip
  inflating: LF-Amazon-1.3M/lbl.json.gz  
  inflating: LF-Amazon-1.3M/trn.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_test.txt  
  inflating: LF-Amazon-1.3M/tst.json.gz  
  inflating: LF-Amazon-1.3M/filter_labels_train.txt  


### Carregando Dataset

In [ ]:
import pandas as pd

#Selecionando parte do dataset para treino rápido
tamanho_da_amostra = 10000
data = pd.read_json('LF-Amazon-1.3M/trn.json.gz', lines=True, nrows=tamanho_da_amostra)
df = pd.DataFrame(data)
print(f"Quantidade total de registros: {len(df)}")
display(df.head())

Quantidade total de registros: 10000


,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
1,0000032034,Adult Ballet Tutu Yellow,,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 16, 33, 36, 37,...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
2,0000913154,The Way Things Work: An Illustrated Encycloped...,,"[116, 117, 118, 119, 120, 121, 122]","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]"
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
4,0001381245,Misty of Chincoteague,,[151],[1.0]


### Limpeza e pré-processamento dos dados

1. Remover os registros que não tenham título ou descrição
2. Remover as colunas que não serão utilizadas
3. Remover caracteres especiais do texto
4. Normalizar o texto
5. Formatar perguntas e respostas
6. Remover linhas duplicadas

In [ ]:
import re
import numpy as np

def normalize_text(text):
    if isinstance(text, str):
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        text = re.sub(r'<.*?>', '', text)
        text = text.replace('\n', ' ').replace('\t', ' ')
        text = re.sub(r'\s+', ' ', text).strip()
        text = text.lower()
    return text

def criar_prompt(row):
    return f"""### Pergunta:Gere uma descrição para o produto com o seguinte título: '{row['title']}'"""

def criar_resposta_esperada(row):
    return f"""### Resposta: {row['content']}"""

def ajustar_prompt(row):
    pergunta = f"[INST] Gere uma descrição para o produto com o seguinte título: '{row['title']}' [/INST]"
    resposta = row['content']
    return f"{pergunta}\n{resposta}"

def ajustar_resposta_esperada(row):
    return f"{row['content']}"

#Removendo as colunas que não serão utilizadas
columns_to_drop = ['target_ind', 'target_rel']
existing_columns_to_drop = [col for col in columns_to_drop if col in df.columns]
if existing_columns_to_drop:
    df = df.drop(existing_columns_to_drop, axis=1)
    print(f"Colunas {existing_columns_to_drop} removidas.")

#Removendo os registros que não tenham título ou descrição
df.replace('', np.nan, inplace=True)
df.dropna(subset=['title', 'content'], inplace=True)

#Removendo registros com títulos duplicados
df.drop_duplicates(subset=['title'], keep='first')

#Normalizando o texto
df['title'] = df['title'].apply(normalize_text)
df['content'] = df['content'].apply(normalize_text)

#Formatando perguntas e respostas
df['text'] = df.apply(ajustar_prompt, axis=1)
df['resposta_esperada'] = df.apply(ajustar_resposta_esperada, axis=1)

print(f"Quantidade total de registros válidos após limpeza do dataset: {len(df)}")
display(df.head())

Quantidade total de registros válidos após limpeza do dataset: 7529


,uid,title,content,text,resposta_esperada
0,0000031909,girls ballet tutu neon pink,high quality 3 layer ballet tutu 12 inches in ...,[INST] Gere uma descrição para o produto com o...,high quality 3 layer ballet tutu 12 inches in ...
3,0001360000,mogs kittens,judith kerr8217s best8211selling adventures of...,[INST] Gere uma descrição para o produto com o...,judith kerr8217s best8211selling adventures of...
7,0000031895,girls ballet tutu neon blue,dance tutu for girls ages 28 years perfect for...,[INST] Gere uma descrição para o produto com o...,dance tutu for girls ages 28 years perfect for...
12,000100039X,the prophet,in a distant timeless place a mysterious proph...,[INST] Gere uma descrição para o produto com o...,in a distant timeless place a mysterious proph...
13,0001473905,rightly dividing the word,this text refers to thepaperbackedition,[INST] Gere uma descrição para o produto com o...,this text refers to thepaperbackedition


### Escolhendo o modelo

- Modelo Mistral 7b: modelo de 7 bilhões de parâmetros conhecido por sua alta capacidade de seguir instruções
- Técnica LoRA: permite carregar e treinar um modelo grande em uma GPU limitada de forma eficiente

In [ ]:
#Login no Hugging Face para baixar o modelo
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

nome_do_modelo = "mistralai/Mistral-7B-Instruct-v0.3"

#Definindo quantização
config_quantizacao = BitsAndBytesConfig(
    load_in_4bit=True,                     # Ativa o carregamento em 4-bit
    bnb_4bit_quant_type="nf4",             # Tipo de quantização (o 'nf4' é uma boa escolha)
    bnb_4bit_compute_dtype=torch.bfloat16, # Tipo de dado para os cálculos
    bnb_4bit_use_double_quant=False,       # Desativa a dupla quantização para economizar um pouco mais de memória
)

#Carregando modelo
modelo = AutoModelForCausalLM.from_pretrained(
    nome_do_modelo,
    quantization_config=config_quantizacao,
    device_map="auto",
)

#Carregando tokenizador
tokenizador = AutoTokenizer.from_pretrained(nome_do_modelo, trust_remote_code=True)
tokenizador.pad_token = tokenizador.eos_token
tokenizador.padding_side = "right"

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

### Teste inicial

In [ ]:
amostra = df.iloc[10]

produto = amostra['title']
resposta_esperada = amostra['content']
prompt_teste = amostra['text']

# Convertendo o prompt em token
inputs = tokenizador(prompt_teste, return_tensors="pt").to("cuda")
outputs = modelo.generate(**inputs, max_new_tokens=256)

# Decodificando a resposta gerada
resposta_gerada = tokenizador.decode(outputs[0], skip_special_tokens=True)

print("="*50)
print(f"PRODUTO SELECIONADO: {produto}")
print(f"RESPOSTA ESPERADA: {resposta_esperada}")
print(f"RESPOSTA DO MODELO ANTES DO FINE-TUNING: {resposta_gerada}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


PRODUTO SELECIONADO: the book of revelation
RESPOSTA ESPERADA: american baptist pastor bible teacher and writer clarence larkin was born october 28 1850 in chester delaware county pennsylvania he was converted to christ at the age of 19 and then felt called to the gospel ministry but the doors of opportunity for study and ministry did not open immediately he then got a job in a bank when he was 21 years old he left the bank and went to college graduating as a mechanical engineer he continued as a professional draftsman for a while then he became a teacher of the blind later failing health compelled him to give up his teaching career after a prolonged rest he became a manufacturer when he was converted he had become a member of the episcopal church but in 1882he became a baptist and was ordained as a baptist minister two years later he went directly from business into the ministry his first charge was at kennett square pennsylvania his second pastorate was at fox chase pennsylvania wher

### Resultado do teste inicial

- Resposta Esperada: Uma descrição sobre uma biografia detalhada de Clarence Larkin, o autor de uma edição específica de um livro sobre o Apocalipse.

- Resposta do Modelo: de acordo com o esperado

- Modelo adicionou texto genérico no final da resposta: *" of 'the book of revelation' by clarence larkin, a comprehensive guide to the biblical book of revelation, featuring detailed charts and explanations of its prophetic truths, dispensational truths, and God's plan and purpose in the ages. This book is a valuable resource for Bible students, pastors, and anyone seeking a deeper understanding of the book of Revelation."*

- Esse resultado veio após um ajuste na formatação do prompt

- Prompt inicial: *"###Pergunta:Gere uma descrição para o produto com o seguinte título: 'the book of revelation'"*





### Executando o Fine Tuning

In [ ]:
from datasets import Dataset
from peft import LoraConfig
from transformers import TrainingArguments
from trl import SFTTrainer

#Convertendo df para dataset do hugging face
dataset_para_treino_pd = df.head(tamanho_da_amostra)
dataset_para_treino_hf = Dataset.from_pandas(dataset_para_treino_pd)
print(f"Treinamento será executado com um Dataset Hugging Face de {len(dataset_para_treino_hf)} registros.")

#Configuração do LoRA
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=8,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
)

#Argumentos de Treinamento GPU A100
#'''
training_arguments = TrainingArguments(
    output_dir="./resultados-treinamento-rapido",
    num_train_epochs=2,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    logging_steps=100,
    save_steps=500,
    save_total_limit=3,
    learning_rate=5e-5,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="none",
)
#'''

#Criação do Treinador
trainer = SFTTrainer(
    model=modelo,
    train_dataset=dataset_para_treino_hf,
    peft_config=peft_config,
    args=training_arguments,
)

print("Iniciando o processo de fine-tuning ===================================")
trainer.train()
print("Fine-tuning concluído! ================================================")

# Salvando modelo treinado
print("Salvando os adaptadores do modelo treinado...")
trainer.model.save_pretrained("modelo-finetuned")

Treinamento será executado com um Dataset Hugging Face de 7529 registros.


Adding EOS to train dataset:   0%|          | 0/7529 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/7529 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/7529 [00:00<?, ? examples/s]

Iniciando o processo de fine-tuning ===================================


Step,Training Loss
100,2.828400
200,2.332600
300,2.256700
400,2.231500
500,2.201600
600,2.176100
700,2.207100
800,2.164700
900,2.144700
1000,2.110300


Fine-tuning concluído! ================================================
Salvando os adaptadores do modelo treinado...


### Reexecução do teste inicial pós treinamento

In [ ]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

#Recarregando modelo base
modelo_base = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-Instruct-v0.3",
    quantization_config=config_quantizacao,
    device_map="auto",
    trust_remote_code=True
)

#Combinando modelo base e treinado
modelo_finetuned = PeftModel.from_pretrained(modelo_base, "./modelo-finetuned")
modelo_finetuned = modelo_finetuned.merge_and_unload()

#Reexecutando teste inicial
amostra = df.iloc[10]
produto = amostra['title']
resposta_esperada = amostra['content']
prompt_teste = amostra['text']
inputs = tokenizador(prompt_teste, return_tensors="pt").to("cuda")

#Gerando resposta com o modelo treinado
outputs = modelo_finetuned.generate(**inputs, max_new_tokens=256)
resposta_gerada_apos_tuning = tokenizador.decode(outputs[0], skip_special_tokens=True)

print("================= RESULTADOS =================")
print(f"PRODUTO: '{produto}'")
print(f"RESPOSTA ESPERADA: {resposta_esperada}")
print(f"RESPOSTA ANTES DO TREINO: {resposta_gerada}")
print(f"RESPOSTA DEPOIS DO TREINO: {resposta_gerada_apos_tuning}")

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/bnb.py:348: UserWarning: Merge lora module to 4-bit linear may get different generations due to rounding errors.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


================= RESULTADOS =================
PRODUTO: 'the book of revelation'
RESPOSTA ESPERADA: american baptist pastor bible teacher and writer clarence larkin was born october 28 1850 in chester delaware county pennsylvania he was converted to christ at the age of 19 and then felt called to the gospel ministry but the doors of opportunity for study and ministry did not open immediately he then got a job in a bank when he was 21 years old he left the bank and went to college graduating as a mechanical engineer he continued as a professional draftsman for a while then he became a teacher of the blind later failing health compelled him to give up his teaching career after a prolonged rest he became a manufacturer when he was converted he had become a member of the episcopal church but in 1882he became a baptist and was ordained as a baptist minister two years later he went directly from business into the ministry his first charge was at kennett square pennsylvania his second pastora

### Comparando resultados

- Resposta Esperada: Uma descrição sobre uma biografia detalhada de Clarence Larkin, o autor de uma edição específica de um livro sobre o Apocalipse.

- Resposta do Modelo: de acordo com o esperado

- Teste com 5000 registros  - Modelo adicionou texto específico no final da resposta: *" of the book of revelation by clarence larkin which is a comprehensive study of the book of revelation and its prophetic significance for the church and the world today."*

- Teste com 10000 registros - Modelo gerou resposta igual à esperada

- Antes do treinamento, o modelo já possuia conhecimento sobre o produto, então gerou um bom resultado. Porém, incluiu texto genérico.

- Depois do treinamento, o modelo permanece conhecendo o produto, e incluiu uma conclusão mais concisa e alinhada com o formato do dataset.

### Teste com outro produto

In [ ]:
import pandas as pd
import random

#Escolhendo item aleatorio para teste
numero_aleatorio = random.randint(0, tamanho_da_amostra-1)
item_teste = df.iloc[numero_aleatorio]
produto_teste = item_teste['title']
resposta_esperada_teste = item_teste['content']

#Gerando resposta com registro não visto no treino
inputs = tokenizador(item_teste['text'], return_tensors="pt").to("cuda")
outputs = modelo_finetuned.generate(**inputs, max_new_tokens=256)
resposta_gerada_teste = tokenizador.decode(outputs[0], skip_special_tokens=True)

print("================= RESULTADOS =================")
print(f"PRODUTO: '{produto_teste}'")
print(f"RESPOSTA ESPERADA: {resposta_esperada_teste}")
print(f"RESPOSTA DEPOIS DO TREINO: {resposta_gerada_teste}")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


================= RESULTADOS =================
PRODUTO: 'house of the hanged'
RESPOSTA ESPERADA: mills has once again delivered an evocatively wellwritten and researched mystery which ticks all the suspense boxes daily mail a mesmerising new novel his best work in an already accomplished career barry forshaw independent mark mills writes beautifully very enjoyable literary review an absorbing thriller and atmospheric plot twister woman home a sumptuously stylish mystery easy living a riveting creepy tale city am outstandingly good beautifully written cleverly plotted and highly recommended literary review on the information officer a forgotten corner of world war ii rediscovered and expertly revealed to us fascinating and shrewdly compelling mark mills does it again william boyd on the information officer a compelling vividly rendered slow burn of a book which culminates in an electrifying climax guardian on the information officer an intriguing puzzle elegantly writtena pleasure to re

### Conclusão

- A técnica de fine-tuning é eficaz para especializar um modelo de linguagem.

- O modelo aprendeu a tarefa designada, gerando descrições coerentes e no estilo desejado.

- Próximos passos: expandir o treinamento para uma porção maior do dataset de 1.3 milhões de registros.